## [The .with_structured_output() method](https://python.langchain.com/docs/how_to/structured_output/#the-with_structured_output-method)

This is the easiest and most reliable way to get structured outputs. `with_structured_output()` is implemented for models that provide native APIs for structuring outputs, like tool/function calling or JSON mode, and makes use of these capabilities under the hood.

This method takes a schema as input which specifies the names, types, and descriptions of the desired output attributes. The method returns a **model-like Runnable**, except that instead of outputting strings or Messages it outputs objects corresponding to the given schema. The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class. **If TypedDict or JSON Schema are used then a dictionary will be returned by the Runnable**, and **if a Pydantic class is used then a Pydantic object will be returned**.

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### Pydantic class
If we want the model to return a Pydantic object, we just need to pass in the desired Pydantic class. The key advantage of using Pydantic is that the model-generated output will be validated. Pydantic will raise an error if any required fields are missing or if any fields are of the wrong type.

In [5]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

res = structured_llm.invoke("Tell me a joke about cats")
print(type(res))
print(res)

<class 'dict'>
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!'}


## [TypedDict or JSON Schema](https://python.langchain.com/docs/how_to/structured_output/#typeddict-or-json-schema)
If you don't want to use Pydantic, explicitly don't want validation of the arguments, or want to be able to stream the model outputs, you can define your schema using a TypedDict class. We can optionally use a special Annotated syntax supported by LangChain that allows you to specify the default value and description of a field. Note, the default value is not filled in automatically if the model doesn't generate it, it is only used in defining the schema that is passed to the model.

```
Requirements
Core: langchain-core>=0.2.26
Typing extensions: It is highly recommended to import Annotated and TypedDict from typing_extensions instead of typing to ensure consistent behavior across Python versions.
```

In [8]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:
    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

ValueError: no signature found for builtin type <class 'dict'>

## [Choosing between multiple schemas](https://python.langchain.com/docs/how_to/structured_output/#choosing-between-multiple-schemas)
The simplest way to let the model choose from multiple schemas is to create a parent schema that has a Union-typed attribute:

In [10]:
from typing import Union


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

In [11]:
structured_llm.invoke("Tell me a joke about cats")

{'final_output': {'setup': 'Why was the cat sitting on the computer?',
  'punchline': 'Because it wanted to keep an eye on the mouse!',
  'rating': 7}}

In [12]:
structured_llm.invoke("what is the capital of France?")

{'final_output': {'response': 'The capital of France is Paris.'}}

## [Few-shot prompting](https://python.langchain.com/docs/how_to/structured_output/#few-shot-prompting)
For more complex schemas it's very useful to add few-shot examples to the prompt. This can be done in a few ways.

The simplest and most universal way is to add examples to a system message in the prompt:

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'final_output': {'setup': 'Woodpecker',
  'punchline': "Woodpecker knock-knocks all day, but can't find a tree that wants to play!",
  'rating': 6}}

When the underlying method for structuring outputs is tool calling, we can pass in our examples as explicit tool calls. You can check if the model you're using makes use of tool calling in its API reference.

In [17]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'final_output': {'setup': 'Knock, knock',
  'punchline': 'Crocodile.',
  'rating': None}}

## [(Advanced) Specifying the method for structuring outputs](https://python.langchain.com/docs/how_to/structured_output/#advanced-specifying-the-method-for-structuring-outputs)
For models that support more than one means of structuring outputs (i.e., they support both tool calling and JSON mode), you can specify which method to use with the `method= argument`.

##### JSON mode
If using JSON mode you'll have to still specify the desired schema in the model prompt. The schema you pass to with_structured_output will only be used for parsing the model outputs, it will not be passed to the model the way it is with tool calling.

To see if the model you're using supports JSON mode, check its entry in the API reference.

In [18]:
structured_llm = llm.with_structured_output(None, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!'}

In [41]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

structured_llm = llm.with_structured_output(Joke, method="json_mode")

prompt = PromptTemplate(
    template="{input}, respond in JSON",
    input_variables=["input"]
)

chain = prompt | structured_llm #| JsonOutputParser(pydantic_object=Joke)

# res = structured_llm.invoke("Tell me a joke about cats, respond in JSON")
res = chain.invoke({"input": "Tell me a joke about cats, respond in JSON"})


print(type(res))
print(res)

<class 'dict'>
{'joke': {'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!'}}


In [39]:
res['joke']

{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!'}

## [(Advanced) Raw outputs](https://python.langchain.com/docs/how_to/structured_output/#advanced-raw-outputs)
LLMs aren't perfect at generating structured output, especially as schemas become complex. You can avoid raising exceptions and handle the raw output yourself by passing `include_raw=True`. This changes the output format to contain the raw message output, the parsed value (if successful), and any resulting errors:

In [19]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7HvVJebKwyUiVrSvKoUjRWzF', 'function': {'arguments': '{"setup":"Why was the cat sitting on the computer?","punchline":"Because it wanted to keep an eye on the mouse!"}', 'name': 'Joke'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 100, 'total_tokens': 129, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-f133ae1c-cfdf-4251-93a5-a8e4bab14982-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!'}, 'id': 'call_7HvVJebKwyUiVrSvKoUjRWzF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens': 29, 'total_tokens': 129}),
 'parsed': {'setup': 'Why was the cat sitting on 

## [Prompting and parsing model outputs directly](https://python.langchain.com/docs/how_to/structured_output/#prompting-and-parsing-model-outputs-directly)
Not all models support `.with_structured_output()`, since not all models have tool calling or JSON mode support. For such models you'll need to directly prompt the model to use a specific format, and use an output parser to extract the structured response from the raw model output.

### [Using PydanticOutputParser](https://python.langchain.com/docs/how_to/structured_output/#using-pydanticoutputparser)
The following example uses the built-in `PydanticOutputParser` to parse the output of a chat model prompted to match the given Pydantic schema. Note that we are adding `format_instructions` directly to the prompt from a method on the parser:

In [20]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [21]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke(query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

In [22]:
chain = prompt | llm | parser

chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

## [Custom Parsing](https://python.langchain.com/docs/how_to/structured_output/#custom-parsing)
You can also create a custom prompt and parser with LangChain Expression Language (LCEL), using a plain function to parse the output from the model:

In [42]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [43]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Output your answer as JSON that  matches the given schema: \`\`\`json
{'$defs': {'Person': {'description': 'Information about a person.', 'properties': {'name': {'description': 'The name of the person', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'The height of the person expressed in meters.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Identifying information about all people in a text.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Make sure to wrap the answer in \`\`\`json and \`\`\` tags
Human: Anna is 23 years old and she is 6 feet tall


In [44]:
chain = prompt | llm | extract_json

chain.invoke({"query": query})

[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]

# OpenAI json_schema
ref: [Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs)

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from typing import Optional
from pydantic import BaseModel, Field

llm = ChatOpenAI(model="gpt-4o-mini")

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

structured_llm = llm.with_structured_output(schema=Joke, method="json_schema")

res = structured_llm.invoke("Tell me a joke about cats")
print(type(res))
print(res)

<class '__main__.Joke'>
setup='Why was the cat sitting on the computer?' punchline='Because it wanted to keep an eye on the mouse!' rating=7


In [10]:
from dotenv import load_dotenv
load_dotenv()

from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from typing import Optional
from pydantic import BaseModel, Field

llm = ChatOpenAI(model="gpt-4o-mini")

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

parser = JsonOutputParser(pydantic_object=Joke)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template ="You are joke master\n {format_instructions}",
                partial_variables={"format_instructions": parser.get_format_instructions()}
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(template="{query}"),
            input_variables=["query"]
        )
    ]
)

chain_1 = prompt | llm | parser

structured_llm = llm.with_structured_output(schema=Joke, method="json_schema")

prompt_2 = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template ="You are joke master.",
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(template="{query}"),
            input_variables=["query"]
        )
    ]
)

chain_2 = prompt_2 | structured_llm


In [13]:
res_1 = chain_1.invoke({"query": "Tell me a joke about cats"})

print(type(res_1))
print(res_1)

res_2 = chain_2.invoke({"query": "Tell me a joke about cats"})

print(type(res_2.dict()))
print(res_2.dict())

<class 'dict'>
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}
<class 'dict'>
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 8}


In [17]:
class Test(BaseModel):
    """Test schema."""
    test: str

test= Test(test="test")

In [14]:
Joke.model_validate(res_2)

Joke(setup='Why did the cat sit on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=8)

In [18]:
Joke.model_validate(test)

ValidationError: 1 validation error for Joke
  Input should be a valid dictionary or instance of Joke [type=model_type, input_value=Test(test='test'), input_type=Test]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type

In [20]:
batch = [{"query": "Tell me a joke about cats"}, {"query": "Tell me a joke about dogs"}, {"query": "Tell me a joke about birds"}]

res_3 = chain_2.batch(batch)
print(res_3)

[Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7), Joke(setup='Why did the dog sit in the shade?', punchline="Because he didn't want to become a hot dog!", rating=7), Joke(setup='Why do seagulls fly over the ocean?', punchline="Because if they flew over the bay, they'd be bagels!", rating=7)]


In [29]:
for r in res_3:
    # print(type(r.dict()))
    # print(type(r))
    test = Joke.model_validate(r.dict())
    print(type(test))
    print(test)

<class '__main__.Joke'>
setup='Why was the cat sitting on the computer?' punchline='Because it wanted to keep an eye on the mouse!' rating=7
<class '__main__.Joke'>
setup='Why did the dog sit in the shade?' punchline="Because he didn't want to become a hot dog!" rating=7
<class '__main__.Joke'>
setup='Why do seagulls fly over the ocean?' punchline="Because if they flew over the bay, they'd be bagels!" rating=7


In [36]:
for r in res_3:
    # print(type(r.dict()))
    # print(type(r))
    dict_r = r.dict()
    # print(dict_r)
    test = Joke(**dict_r)
    print(type(test))
    print(test)

<class '__main__.Joke'>
setup='Why was the cat sitting on the computer?' punchline='Because it wanted to keep an eye on the mouse!' rating=7
<class '__main__.Joke'>
setup='Why did the dog sit in the shade?' punchline="Because he didn't want to become a hot dog!" rating=7
<class '__main__.Joke'>
setup='Why do seagulls fly over the ocean?' punchline="Because if they flew over the bay, they'd be bagels!" rating=7
